#  Example with Stanford SR830 Bundle

In [1]:
# IMPORTS

#%matplotlib notebook
import qcodes as qc
import os
import numpy as np
from time import sleep
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from SR830_ext import SR830_ext
from bundle_lockin import BundleLockin
from qcodes.instrument.base import Instrument
from qcodes.utils.validators import Numbers, Arrays
from qcodes.utils.dataset import doNd


from qcodes import initialise_or_create_database_at, load_or_create_experiment, load_by_id
tutorial_db_path = os.path.join(os.getcwd(), 'tutorial_doND.db')
initialise_or_create_database_at(tutorial_db_path)
load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#1@C:\Users\Farzad\Documents\Rasmus\SR830ext\tutorial_doND.db
-----------------------------------------------------------------------------------
1-results-1-lockin_setpoints,lockin_trace-1
2-results-2-gen2_v_now,lockin_setpoints,lockin_trace-20
3-results-3-lockin_setpoints,lockin_trace-1
4-results-4-gen2_v_now,lockin_setpoints,lockin_trace-20
5-results-5-lockin_setpoints,lockin_trace-0
6-results-6-lockin_setpoints,lockin_trace-0
7-results-7-lockin_setpoints,lockin_trace-1
8-results-8-gen2_v_now,lockin_setpoints,lockin_trace-20
9-results-9-lockin_setpoints,lockin_trace-1
10-results-10-lockin_setpoints,lockin_trace-1
11-results-11-gen2_v_now,lockin_setpoints,lockin_trace-20
12-results-12-lockin_setpoints,lockin_trace-1
13-results-13-gen2_v_now,lockin_setpoints,lockin_trace-20

In [2]:
class DummyGenerator(Instrument):

    def __init__(self, name, **kwargs):

        super().__init__(name, **kwargs)

        self.add_parameter('v_start',
                           initial_value=0,
                           unit='V',
                           label='v start',
                           vals=Numbers(0,1e3),
                           get_cmd=None,
                           set_cmd=None)

        self.add_parameter('v_stop',
                           initial_value=1,
                           unit='V',
                           label='v stop',
                           vals=Numbers(1,1e3),
                           get_cmd=None,
                           set_cmd=None)
        
        self.add_parameter('v_now',
                           initial_value=0,
                           unit='V',
                           label='v_now',
                           vals=Numbers(self.v_start(),self.v_stop()),
                           get_cmd=None,
                           set_cmd=None)

In [3]:
slow = DummyGenerator('slow')
fast = DummyGenerator('fast')

In [4]:
# CONNECT TO INSTRUMENT

sr = SR830_ext('lockin', 'GPIB0::1::INSTR')

Connected to: Stanford_Research_Systems SR830 (serial:s/n40974, firmware:ver1.07) in 0.97s


In [ ]:
sr.ch1_display('X')
sr.ch1_ratio('none')
sr.buffer_SR('Trigger')
sr.buffer_trig_mode.set('ON')
sr.set_sweep_parameters(gen.v_now, 0,0.5,20,0.2, label="Voltage")

In [ ]:
lockins = (sr,)
bundle = BundleLockin('bundle',lockins)


In [ ]:
bundle.set_sweep_parameters(gen2.v_now, 0,0.5,20, label="Voltage")

In [ ]:
sr.buffer_reset()
for x in np.linspace(0,0.5,20):
    gen2.v_now.set(x)
    sleep(0.1)
    sr.send_trigger()
    sr.ch1_databuffer.prepare_buffer_readout()    
 

In [ ]:
bundle.trace_lockin.get()

In [ ]:
bundle_parameters = bundle.__dict__['parameters']
traces = [b[key] for key in b.keys() if 'trace' in key]

In [ ]:
lockins[0].name

In [ ]:
meas = Measurement()
refresh_time = 1. # in s
delay_fast = 0.1
delay_slow = 0.1
num_points_fast = 20
num_points_slow = 20
start_slow = 0
end_slow = 0.5
start_fast = 0
end_fast = 0.5

bundle.set_sweep_parameters(gen2.v_now, start_fast,end_fast,num_point_fast, label="Voltage")
interval_slow = np.linspace(start_slow,end_slow,num_points_slow)
meas.write_period = refresh_time
set_points_fast = bundle.setpoints

meas.register_parameter(set_point_fast)
fast.post_delay = delay_fast
 
meas.register_parameter(slow)
slow.post_delay = delay_slow


bundle_parameters = bundle.__dict__['parameters']
traces = [b[key] for key in b.keys() if 'trace' in key]
for trace in traces:
        meas.register_parameter(trace, setpoints=(set_point_slow, set_point_fast))

progress_bar = progressbar.ProgressBar(max_value=num_points_slow * num_points_fast)
points_taken = 0
time.sleep(0.1)

    with meas.run(write_in_background=True) as datasaver:
        run_id = datasaver.run_id
        last_time = time.time()
        for point_slow in interval_slow:
            #param_set2.set(start2)
            slow.set(point_slow)
            #outputs = []
        
            for lockin in bundle.lockins:
                lockin.buffer_reset()
                
            for point_fast in set_points2:
                param_set2.set(point)
                for lockin in param_meas:
                    lockin.send_trigger()
        data = []
        for trace in traces:   
            data.append((trace,trace.get()))
        data.append((slow,slow.get()),)

                # notice that output is a list of tuples and it is created from scratch at each new iteration                
                output = ([(param_set1, set_point1), (param_set2, set_point2)] +
                          [(parameter, parameter.get()) for parameter in param_meas])
                outputs.append(output)
                points_taken += 1
            for output in outputs:
                datasaver.add_result(*output)
                # current_time = time.time()
                # if current_time - last_time >= refresh_time:
                #     last_time = current_time
                #     progress_bar.update(points_taken)
            progress_bar.update(points_taken)
    return run_id


## List of parameters

In [ ]:
doNd.do0d(sr.trace,do_plot=True)

In [ ]:
doNd.do1d(gen2.v_now, 0.01,1.0,20,1,sr.trace,do_plot=True)